## Introduction and set up

If you've finished going through my [Linear model and neural net from scratch
](https://www.kaggle.com/code/jhoward/linear-model-and-neural-net-from-scratch) notebook, then now is a good time to look at how to do the same thing using a library, instead of doing it from scratch. We'll use fastai and PyTorch. The benefits of using these libraries is:

- Best practices are handled for you automatically -- fast.ai has done thousands of hours of experiments to figure out what the best settings are for you
- Less time getting set up, which means more time to try out your new ideas
- Each idea you try will be less work, because fastai and PyTorch will do the many of the menial bits for you
- You can always drop down from fastai to PyTorch if you need to customise any part (or drop down from the fastai Application API to the fastai mid or low tier APIs), or even drop down from PyTorch to plain python for deep customisation.

Let's see how that looks in practice. We'll start by doing the same library setup as in the "from scratch" notebook:

In [32]:
from pathlib import Path
import os

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/titanic')
    !pip install -Uqq fastai
else:
    import zipfile,kaggle
    path = Path('titanic')
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


We'll import the fastai tabular library, set a random seed so the notebook is reproducible, and pick a reasonable number of significant figures to display in our tables:

In [33]:
from fastai.tabular.all import *

pd.options.display.float_format = '{:.2f}'.format
'''将浮点数的显示格式设置为保留两位小数'''
set_seed(42)

In [34]:
# from fastai.tabular.all import *
# pd.options.display.float_format = f'{:.2f}'
# set_seed(42)

## Prep the data

We'll read the CSV file just like we did before:

In [35]:
df = pd.read_csv(path/'train.csv')

When you do everything from scratch, every bit of feature engineering requires a whole lot of work, since you have to think about things like dummy variables, normalization, missing values, and so on. But with fastai that's all done for you. So let's go wild and create lots of new features! We'll use a bunch of the most interesting ones from this fantastic [Titanic feature engineering notebook](https://www.kaggle.com/code/gunesevitan/titanic-advanced-feature-engineering-tutorial/) (and be sure to click that link and upvote that notebook if you like it to thank the author for their hard work!)

In [36]:
def add_features(df):
    df['LogFare'] = np.log1p(df['Fare'])
    df['Deck'] = df.Cabin.str[0].map(dict(A="ABC", B="ABC", C="ABC", D="DE", E="DE", F="FG", G="FG"))
    df['Family'] = df.SibSp+df.Parch
    df['Alone'] = df.Family==1
    df['TicketFreq'] = df.groupby('Ticket')['Ticket'].transform('count')
    ''' 使用 groupby 函数按照 'Ticket' 列的值对 DataFrame 进行分组 
    使用 transform('count') 方法，对每个乘客的 'Ticket' 列进行计数操作
    '''
    df['Title'] = df.Name.str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
    '''先用逗号划分,如'Braund, Mr. Owen Harris',再分为2列,再把第二列划分,把象征男女的Mr,Miss,Master等分离出来成为一列
    这里注意划分的时候要用 逗号＋空格 ,不然Mr前面就有个空格，下一行代码无法执行
    '''
    df['Title'] = df.Title.map(dict(Mr="Mr",Miss="Miss",Mrs="Mrs",Master="Master")).value_counts(dropna=False)
    '''成年男性，未成年男性，未婚女性，已婚女性'''
add_features(df)

In [110]:
def add_features(df):
    df['LogFare']  = np.log1p(df['Fare'])
    df['Deck'] = df.Cabin.str[0].map(dict(A='ABC',B='ABC',C='ABC',D='DE',E='DE',F='FG',G='FG'))
    df['Family'] = df['SibSp']+df['Parch']
    df['Alone'] = df.Family==1
    df['TicketFreq'] = df.groupby('Ticket')['Ticket'].transform('count')
    df['Title'] = df.Name.str.split(', ',expand=True)[1].str.split('.',expand = True)[0]
    df['Title'] = df.Title.map(dict(Mr='Mr',Miss='Miss',Mrs = 'Mrs',Master = 'Master'))
add_features(df)

As we discussed in the last notebook, we can use `RandomSplitter` to separate out the training and validation sets:

In [185]:
splits = RandomSplitter(seed=42)(df)
splits

((#713) [788,525,821,253,374,98,215,313,281,305...],
 (#178) [303,778,531,385,134,476,691,443,386,128...])

Now the entire process of getting the data ready for training requires just this one cell!:

In [ ]:
dls = TabularPandas(
    df, splits=splits,
    procs = [Categorify, FillMissing, Normalize],
    cat_names=["Sex","Pclass","Embarked","Deck", "Title"],
    cont_names=['Age', 'SibSp', 'Parch', 'LogFare', 'Alone', 'TicketFreq', 'Family'],
    y_names="Survived", y_block = CategoryBlock(),
).dataloaders(path=".")

In [118]:
dls = TabularPandas(
    df,splits=splits,
    procs=[Categorify,FillMissing,Normalize],
    cat_names=["Sex","Pclass","Embarked","Deck", "Title"],
    cont_names=['Age', 'SibSp', 'Parch', 'LogFare', 'Alone', 'TicketFreq', 'Family'],
    y_names='Survived',  #  预测目标的列名
    y_block=CategoryBlock()
).dataloaders(path='.')# 表示数据的路径为当前工作目录

c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\data\transforms.py:225: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(col):
c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\tabular\core.py:233: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(c):
c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\tabular\core.py:233: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(c):
c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\tabular\core.py:233: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
 

Here's what each of the parameters means:

- Use `splits` for indices of training and validation sets:

      splits=splits,
    
- Turn strings into categories, fill missing values in numeric columns with the median, normalise all numeric columns:
    
      procs = [Categorify, FillMissing, Normalize],
    
- These are the categorical independent variables:
    
      cat_names=["Sex","Pclass","Embarked","Deck", "Title"],
    
- These are the continuous independent variables:
    
      cont_names=['Age', 'SibSp', 'Parch', 'LogFare', 'Alone', 'TicketFreq', 'Family'],
    
- This is the dependent variable:
    
      y_names="Survived",

- The dependent variable is categorical (so build a classification model, not a regression model):

      y_block = CategoryBlock(),

## Train the model

The data and model together make up a `Learner`. To create one, we say what the data is (`dls`), and the size of each hidden layer (`[10,10]`), along with any metrics we want to print along the way:

In [ ]:
learn = tabular_learner(dls, metrics=accuracy, layers=[10,10])

In [119]:
learn = tabular_learner(dls,metrics=accuracy,layers=[10,10])

You'll notice we didn't have to do any messing around to try to find a set of random coefficients that will train correctly -- that's all handled automatically.

One handy feature that fastai can also tell us what learning rate to use:

In [ ]:
learn.lr_find(suggest_funcs=(slide, valley))

SuggestedLRs(slide=0.03981071710586548, valley=0.013182567432522774)

In [120]:
learn.lr_find(suggest_funcs=(slide,valley))
'''
slide: 用于寻找学习率曲线上最陡峭的下降点，通常表示在损失函数下降最快的学习率。
这个值通常是一个比较大的学习率，处于损失函数下降最为迅猛的位置。
valley: 用于寻找学习率曲线上的谷底，表示学习率处于能够最快收敛到最小损失的位置。
这个值通常是一个相对较小的学习率。
figure:y纵坐标显示的是在每个学习率下训练模型的结果
'''

The two colored points are both reasonable choices for a learning rate. I'll pick somewhere between the two (0.03) and train for a few epochs:

In [ ]:
learn.fit(16, lr=0.03)

In [121]:
learn.fit(n_epoch=16,lr=0.03)

epoch,train_loss,valid_loss,accuracy,time
0,0.566168,0.604746,0.617977,00:00
1,0.511843,0.516013,0.764045,00:00
2,0.476778,0.454039,0.825843,00:00
3,0.453338,0.415937,0.825843,00:00
4,0.436682,0.409475,0.814607,00:00
5,0.422505,0.403506,0.820225,00:00
6,0.414106,0.407416,0.837079,00:00
7,0.403823,0.404409,0.831461,00:00
8,0.403148,0.413958,0.808989,00:00
9,0.401314,0.426014,0.820225,00:00


We've got a similar accuracy to our previous "from scratch" model -- which isn't too surprising, since as we discussed, this dataset is too small and simple to really see much difference. A simple linear model already does a pretty good job. But that's OK -- the goal here is to show you how to get started with deep learning and understand how it really works, and the best way to do that is on small and easy to understand datasets.

## Submit to Kaggle

One important feature of fastai is that all the information needed to apply the data transformations and the model to a new dataset are stored in the learner. You can call `export` to save it to a file to use it later in production, or you can use the trained model right away to get predictions on a test set.

To submit to Kaggle, we'll need to read in the test set, and do the same feature engineering we did for the training set:

In [129]:
tst_df = pd.read_csv(path/'test.csv')
tst_df['Fare'] = tst_df.Fare.fillna(0)
add_features(tst_df)

In [135]:
tst_df = pd.read_csv(path/'test.csv')
tst_df['Fare'] = tst_df['Fare'].fillna(0)
add_features(tst_df)

In [136]:
tst_df.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
LogFare          0
Deck           327
Family           0
Alone            0
TicketFreq       0
Title            7
dtype: int64

But we don't need to manually specify any of the processing steps necessary to get the data ready for modeling, since that's all saved in the learner. To specify we want to apply the same steps to a new dataset, use the `test_dl()` method:

In [137]:
tst_dl = learn.dls.test_dl(tst_df)

c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\tabular\core.py:233: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(c):
c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\tabular\core.py:233: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(c):
c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\tabular\core.py:233: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(c):
c:\Users\lenovo\miniconda3\envs\fastai\lib\site-packages\fastai\tabular\core.py:233: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  

In [ ]:
tst_dl = learn.dls.test_dl(tst_df)

Now we can use `get_preds` to get the predictions for the test set:

In [143]:
preds,_ = learn.get_preds(dl=tst_dl)

In [ ]:
preds,_ = learn.get_preds(dl=tst_dl)

In [160]:
type(_)

NoneType

Finally, let's create a submission CSV just like we did in the previous notebook...

In [ ]:
tst_df['Survived'] = (preds[:,1]>0.5).int()
'''
因为对于二分类问题，模型通常返回两列预测结果。如果使用了softmax或sigmoid激活函数，
通常第一列是代表类别 0 的预测概率，而第二列是代表类别 1 的预测概率。
比如第一行(tensor(0.9042), tensor(0.0958))相加得1
'''
sub_df = tst_df[['PassengerId','Survived']]
sub_df.to_csv('sub.csv', index=False)

In [158]:
tst_df['Survived'] = (preds[:,1]>0.5).int()
sub_df = tst_df[['PassengerId','Survived']]
sub_df.to_csv('sub.csv',index=False)

In [157]:
(preds[:,1]>0.5).float()

tensor([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
        0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
        1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
        0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
        0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 

...and check that it looks reasonable:

In [159]:
!head sub.csv

'head' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


## Ensembling

Since it's so easy to create a model now, it's easier to play with more advanced modeling approaches. For instance, we can create five separate models, each trained from different random starting points, and average them. This is the simplest approach of [ensembling](https://machinelearningmastery.com/tour-of-ensemble-learning-algorithms/) models, which combines multiple models to generate predictions that are better than any of the single models in the ensemble.

To create our ensemble, first we copy the three steps we used above to create and train a model, and apply it to the test set:

In [ ]:
learn.no_mbar

In [ ]:
def ensemble():
    learn = tabular_learner(dls, metrics=accuracy, layers=[10,10])
    with learn.no_bar(),learn.no_logging(): learn.fit(16, lr=0.03)
    ''' learn.no_bar() 和 learn.no_logging() 用于在训练期间禁用训练进度条和日志记录 '''
    return learn.get_preds(dl=tst_dl)[0]

In [168]:
def ensemble():
    learn = tabular_learner(dls,metrics=accuracy,layers=[10,10])
    with learn.no_logging(),learn.no_bar(): learn.fit(16,0.03)
    return learn.get_preds(dl=tst_dl)[0]

Now we run this five times, and collect the results into a list:

In [169]:
learns = [ensemble() for _ in range(5)]

In [166]:
learns = [ensemble() for i in range(5)]

We stack this predictions together and take their average predictions:

In [172]:
len(learns),len(preds)

(5, 418)

In [176]:
len(torch.stack(learns))

5

In [179]:
type(torch.stack(learns))

torch.Tensor

In [182]:
torch.stack(learns).shape

torch.Size([5, 418, 2])

In [183]:
ens_preds = torch.stack(learns).mean(0)
''' stack(learns) 将这些张量按照给定维度进行堆叠，生成一个新的张量。默认情况下，这会在新的维度（0号维度）上进行堆叠 '''
ens_preds.shape

torch.Size([418, 2])

Finally, use the same code as before to generate a submission file, which we can submit to Kaggle after the notebook is saved and run:

In [ ]:
tst_df['Survived'] = (ens_preds[:,1]>0.5).int()
sub_df = tst_df[['PassengerId','Survived']]
sub_df.to_csv('ens_sub.csv', index=False)

In [184]:
tst_df['Survived'] = (ens_preds[:,1]>0.5).int()
sub_df = tst_df[['PassengerId','Survived']]
sub_df.to_csv('ens_sub.csv',index=False)

At the time of writing, this submission is well within the top 25% of entries to the competition.

(A lot of submissions to this competition use additional external data, but we have restricted ourselves to just using the data provided. We'd probably do a lot better if we used external data too. Feel free to give that a try, and see how you go. Note that you'll never be able to get to the top of the leaderboard, since a lot of folks in this competition have cheated, by downloading the answers from the internet and uploading them as their submission. In a real competition that's not possible, because the answers aren't public, but there's nothing stopping people from cheating in a tutorial/practice competition like this one. So if you're ready for a real challenge, take a look at the [competitions page](https://www.kaggle.com/competitions/) and start working on a real competition!)

## Final thoughts

As you can see, using fastai and PyTorch made things much easier than doing it from scratch, but it also hid away a lot of the details. So if you only ever use a framework, you're not going to as fully understand what's going on under the hood. That understanding can be really helpful when it comes to debugging and improving your models. But do use fastai when you're creating models on Kaggle or in "real life", because otherwise you're not taking advantage of all the research that's gone into optimising the models for you, and you'll end up spending more time debugging and implementing menial boiler-plate than actually solving the real problem!

If you found this notebook useful, please remember to click the little up-arrow at the top to upvote it, since I like to know when people have found my work useful, and it helps others find it too. (BTW, be sure you're looking at my [original notebook](https://www.kaggle.com/jhoward/why-you-should-use-a-framework) here when you do that, and are not on your own copy of it, otherwise your upvote won't get counted!) And if you have any questions or comments, please pop them below -- I read every comment I receive!